### Generalized Odd-Even Sort with Semaphores (Python) [4 points]

In odd-even sort, pairs of consecutive elements are swapped in phases: First, the pairs at indices (0, 1), (2, 3), (4, 5), ... are swapped if necessary, then those at indices (1, 2), (3, 4), (5, 6), ..., then this is repeated until all elements are sorted:

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | index   |
| - | - | - | - | - | - | - | - | - | - |:------- |
| 8 | 1 | 7 | 0 | 4 | 5 | 3 | 9 | 6 | 2 | initial |
| 1 | 8 | 0 | 7 | 4 | 5 | 3 | 9 | 2 | 6 | swapping at even-odd indices |
| 1 | 0 | 8 | 4 | 7 | 3 | 5 | 2 | 9 | 6 | swapping at odd-even indices |
| 0 | 1 | 4 | 8 | 3 | 7 | 2 | 5 | 6 | 9 | swapping at even-odd indices |
| 0 | 1 | 4 | 3 | 8 | 2 | 7 | 5 | 6 | 9 | swapping at odd-even indices |
| 0 | 1 | 3 | 4 | 2 | 8 | 5 | 7 | 6 | 9 | swapping at even-odd indices |
| 0 | 1 | 3 | 2 | 4 | 5 | 8 | 6 | 7 | 9 | swapping at odd-even indices |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 8 | 7 | 9 | swapping at even-odd indices |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | swapping at odd-even indices |

Like bubble sort, the algorithm is based on swapping (sorting) consecutive elements. Here, all swaps in one phase can be done in parallel. If there are `N` elements, approximately `N / 2` processes can swap elements in parallel. If `N` is large, that would lead to a prohibitive number of processes. In the generalized odd-even sort, each process does not sort two consecutive elements but a subsequence. Suppose two processes are sorting the 10 elements above. In the first phase, one process sorts at indices 0 .. 3 and the other at 4 .. 7. In the second phase, one process sorts at indices 2 .. 5 and the other at 6 .. 9. Then, this is repeated until all elements are sorted:

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | index   |
| - | - | - | - | - | - | - | - | - | - |:------- |
| 8 | 1 | 7 | 0 | 4 | 5 | 3 | 9 | 6 | 2 | initial |
| 0 | 1 | 7 | 8 | 3 | 4 | 5 | 9 | 6 | 2 | sorting 0 .. 3 and 4 .. 7 |
| 0 | 1 | 3 | 4 | 7 | 8 | 2 | 5 | 6 | 9 | sorting 2 .. 5 and 6 .. 9 |
| 0 | 1 | 3 | 4 | 2 | 5 | 7 | 8 | 6 | 9 | sorting 0 .. 3 and 4 .. 7 |
| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | sorting 2 .. 5 and 6 .. 9 |

Generalizing this to `N` elements and `P` processes, each process sorts approximately `N / P` elements sequentially. In the implementation below, quicksort is used for sequential sorting. Each round consists of two sorting phases. For simplicity, the array and the barriers are kept as global variables. This Python implementation is not going to be faster than a sequential implementation due to the [global interpreter lock](https://docs.python.org/3/glossary.html#term-global-interpreter-lock), but it illustrates the point. To simplify the index calculations, `oddevensort` computes the list size to be sorted from the number of threads and the number of elements each thread sorts. A random list is then generated. You can uncomment some lines with test output to see the program work, but be aware that the output may be asynchronous.

In [6]:
from threading import Thread, Semaphore
from time import sleep
from sys import stdout
from random import random, shuffle

class Sorter(Thread):
    def __init__(self, l0, u0, l1, u1, i, r):
        # stdout.write("Sorter " + str(i) + " init " + str(l0) + " " + str(u0) + " " + \
        #             str(l1) + " " + str(u1) + "\n")
        Thread.__init__(self)
        self.l0, self.u0, self.l1, self.u1 = l0, u0, l1, u1
        self.i, self.r = i, r

    def partition(self, p, r):
        global a
        x, i = a[r], p - 1;
        for j in range(p, r):
            if a[j] <= x: i += 1; a[i], a[j] = a[j], a[i]
        a[i + 1], a[r] = a[r], a[i + 1]
        return i + 1

    def sequentialsort(self, p, r):
        if p < r:
            q = self.partition(p, r)
            self.sequentialsort(p, q - 1)
            self.sequentialsort(q + 1, r)

    def barriersync(self):
        for j in range (len(barriers)): 
            if j != self.i: barriers[j].release()
        for _ in range (len(barriers)-1): barriers[self.i].acquire()

    def run(self):
        for r in range(self.r):
            stdout.write(str(self.i) + " sorts " + str(self.l0) + " to " + str(self.u0) + " round " + str(r) + "\n")
            self.sequentialsort(self.l0, self.u0)
            #sleep(random())
            self.barriersync()
            stdout.write(str(self.i) + " sorts " + str(self.l1) + " to " + str(self.u1) + " round " + str(r) + "\n")
            self.sequentialsort(self.l1, self.u1)
            #sleep(random())
            self.barriersync()

def oddevensort(P, M):
    # P: number of sorting threads
    # M: number of elements each thread sorts sequentially
    N = P * M + M // 2 # number of elements to be sorted
    print("size", N)
    global a; a = list(range(N)); shuffle(a)
    # print("init", a)
    # initialization of (global) barriers
    global barriers; barriers = []
    for _ in range(P): barriers.append(Semaphore(0))
        
    sorters = []
    for i in range(P):
        s = Sorter(i * M, (i + 1) * M - 1, i * M + M // 2, (i + 1) * M + M // 2 - 1, i, P + 1)
        s.start(); sorters.append(s)
    for s in sorters: s.join()
    print("sorted", a)
    assert a == list(range(N))

In [7]:
oddevensort(2, 10)

size 25
0 sorts 0 to 9 round 0
1 sorts 10 to 19 round 0
1 sorts 15 to 24 round 0
0 sorts 5 to 14 round 0
0 sorts 0 to 9 round 1
1 sorts 10 to 19 round 1
1 sorts 15 to 24 round 1
0 sorts 5 to 14 round 1
0 sorts 0 to 9 round 2
1 sorts 10 to 19 round 2
1 sorts 15 to 24 round 2
0 sorts 5 to 14 round 2
sorted [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [8]:
oddevensort(4, 10)

size 45
0 sorts 0 to 9 round 0
1 sorts 10 to 19 round 0
2 sorts 20 to 29 round 0
3 sorts 30 to 39 round 0
3 sorts 35 to 44 round 0
0 sorts 5 to 14 round 0
2 sorts 25 to 34 round 0
1 sorts 15 to 24 round 0
1 sorts 10 to 19 round 1
0 sorts 0 to 9 round 1
2 sorts 20 to 29 round 1
3 sorts 30 to 39 round 1
3 sorts 35 to 44 round 1
1 sorts 15 to 24 round 1
2 sorts 25 to 34 round 1
0 sorts 5 to 14 round 1
0 sorts 0 to 9 round 2
2 sorts 20 to 29 round 2
3 sorts 30 to 39 round 2
1 sorts 10 to 19 round 2
1 sorts 15 to 24 round 2
2 sorts 25 to 34 round 2
0 sorts 5 to 14 round 2
3 sorts 35 to 44 round 2
3 sorts 30 to 39 round 3
0 sorts 0 to 9 round 3
2 sorts 20 to 29 round 3
1 sorts 10 to 19 round 3
1 sorts 15 to 24 round 3
3 sorts 35 to 44 round 3
2 sorts 25 to 34 round 3
0 sorts 5 to 14 round 3
0 sorts 0 to 9 round 4
2 sorts 20 to 29 round 4
1 sorts 10 to 19 round 4
3 sorts 30 to 39 round 4
3 sorts 35 to 44 round 4
0 sorts 5 to 14 round 4
2 sorts 25 to 34 round 4
1 sorts 15 to 24 round 4
sorted 